# Character level generator

1. Take a continuous flow of characters
2. split it in chunks of SEQ_LENGTH
3. evaluate labels by moving sequence by 1 to right

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import tensorflow as tf

In [6]:
# DIVINA COMMEDIA
path_to_file = '/content/drive/My Drive/DATASETS/DANTE_DIVINA_COMMEDIA/la_divin.txt'
encoding = 'latin-1'
subfolder = 'seq2seq_divina'

In [7]:
CHECKPOINT_DIR = os.path.join('/content/drive/My Drive/colab',subfolder,'training_checkpoints')
SAVE_DIR = os.path.join('/content/drive/My Drive/colab',subfolder,'saved_model')
METADATA_DIR = os.path.join('/content/drive/My Drive/colab',subfolder,'metadata')

In [8]:
text = open(path_to_file, 'rb').read().decode(encoding=encoding)

In [9]:
# collect all chars
import pickle

alphabet = list(set(text))

if not os.path.exists(METADATA_DIR):
    os.makedirs(METADATA_DIR)
    
with open(os.path.join(METADATA_DIR, 'alphabet.pkl'), 'wb') as f:
  pickle.dump(alphabet, f)
# TODO: save alphabet with order (to recover char2ind and ind2char)

In [10]:
def make_mappings(alphabet):

  char2ind = {ch:ind for ind, ch in enumerate(alphabet)}
  ind2char = {ind:char for ind, char in enumerate(alphabet)}

  return alphabet, char2ind, ind2char

In [11]:
with open(os.path.join(METADATA_DIR, 'alphabet.pkl'), 'rb') as f:
    al, char2ind, ind2char = make_mappings(pickle.load(f))

In [12]:
for ch in char2ind.keys():
    assert ind2char[char2ind[ch]] == ch

for ind in ind2char.keys():
    assert char2ind[ind2char[ind]] == ind


In [13]:
# map input to a list of integer symbols
text_as_int = np.array([char2ind[c] for c in text])

In [14]:
# list mapping of first characters
# TEST
print('{')
for char,_ in zip(char2ind, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2ind[char]))
print('  ...\n}')

{
  'é' :   0,
  '.' :   1,
  '-' :   2,
  'x' :   3,
  'D' :   4,
  's' :   5,
  'Q' :   6,
  '«' :   7,
  'U' :   8,
  '\x85':   9,
  'v' :  10,
  'í' :  11,
  'o' :  12,
  'ú' :  13,
  'G' :  14,
  'Ï' :  15,
  'ë' :  16,
  'F' :  17,
  'y' :  18,
  'ò' :  19,
  ...
}


# Create the dataset
- each example is a sequence of SEQ_LENGTH + 1 elements 

In [15]:
SEQ_LENGTH = 100
EXAMPLES_PER_EPOCH = len(text) // SEQ_LENGTH

char_dset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dset.batch(SEQ_LENGTH+1, drop_remainder=True)

In [16]:
# TEST
for item in sequences.take(5):
    print(repr(''.join([ ind2char[ind] for ind in item.numpy() ])))

'Dante Alighieri\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nLA DIVINA COMMEDIA\r\n\r\nINFERNO\r\n\r\n\r\n\r\n\r\n\r\n\r\nCANTO PRIMO\r\n\r\n  Nel me'
'zzo del cammin di nostra vita\r\nmi ritrovai per una selva oscura\r\nché la diritta via era smarrita.\r\n  '
'Ah quanto a dir qual era è cosa dura\r\nesta selva selvaggia e aspra e forte\r\nche nel pensier rinova la'
" paura!\r\n  Tant'è amara che poco è piú morte;\r\nma per trattar del ben ch'io vi trovai,\r\ndirò dell'alt"
"re cose ch'i' v'ho scorte.\r\n  Io non so ben ridir com'io v'entrai,\r\ntant'era pieno di sonno a quel pu"


In [17]:
def split_input_target(chunk):
  """
  From a common sequence, generate input and target

  :return:
    input_text: elements from start to end-1
    target_text: elements from start+1 to end
  """
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
# TEST
for inputs, targets in dataset.take(5):
    print('in: {}'.format(repr(''.join([ ind2char[ind] for ind in inputs.numpy() ]))))
    print('tg: {}\n'.format(repr(''.join([ ind2char[ind] for ind in targets.numpy() ]))))

in: 'Dante Alighieri\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nLA DIVINA COMMEDIA\r\n\r\nINFERNO\r\n\r\n\r\n\r\n\r\n\r\n\r\nCANTO PRIMO\r\n\r\n  Nel m'
tg: 'ante Alighieri\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nLA DIVINA COMMEDIA\r\n\r\nINFERNO\r\n\r\n\r\n\r\n\r\n\r\n\r\nCANTO PRIMO\r\n\r\n  Nel me'

in: 'zzo del cammin di nostra vita\r\nmi ritrovai per una selva oscura\r\nché la diritta via era smarrita.\r\n '
tg: 'zo del cammin di nostra vita\r\nmi ritrovai per una selva oscura\r\nché la diritta via era smarrita.\r\n  '

in: 'Ah quanto a dir qual era è cosa dura\r\nesta selva selvaggia e aspra e forte\r\nche nel pensier rinova l'
tg: 'h quanto a dir qual era è cosa dura\r\nesta selva selvaggia e aspra e forte\r\nche nel pensier rinova la'

in: " paura!\r\n  Tant'è amara che poco è piú morte;\r\nma per trattar del ben ch'io vi trovai,\r\ndirò dell'al"
tg: "paura!\r\n  Tant'è amara che poco è piú morte;\r\nma per trattar del ben ch'io vi trovai,\r\ndirò dell'alt"

in: "re cose ch'i' v'ho 

In [20]:
# configuration
BATCH_SIZE = 64
BUFFER_SIZE = 10000
RNN_UNITS = 1024
EMBEDDING_DIM = 256
VOCAB_SIZE = len(alphabet)

In [21]:

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [22]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [24]:
model =  build_model(
    vocab_size = VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNITS,
    batch_size=BATCH_SIZE)

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           19968     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 78)            79950     
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 78) # (batch_size, sequence_length, vocab_size)


In [27]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [28]:
print('in: {}'.format(repr(''.join([ ind2char[ind] for ind in sampled_indices ]))))

in: "ümfFìÏ\x85dìB'gèf!\x85èÀíbë\riSoUabm-ýLxàÀRH'EcGDO[pTeZàúE[vJNÀ(qL)ésUeVìòR.àéAVDEBttoúbFúqrd\x85RsÏOzQýF(\nhlü"


In [29]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 78)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.356991


In [30]:
model.compile(optimizer='adam', loss=loss)

In [31]:
# Name of the checkpoint files
checkpoint_prefix = os.path.join(CHECKPOINT_DIR, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [32]:
EPOCHS=100

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
85/85 [==============================] - 6s 56ms/step - loss: 3.5881
Epoch 2/100
85/85 [==============================] - 5s 56ms/step - loss: 2.1782
Epoch 3/100
85/85 [==============================] - 5s 57ms/step - loss: 1.9672
Epoch 4/100
85/85 [==============================] - 5s 57ms/step - loss: 1.8380
Epoch 5/100
85/85 [==============================] - 6s 58ms/step - loss: 1.7413
Epoch 6/100
85/85 [==============================] - 6s 60ms/step - loss: 1.6550
Epoch 7/100
85/85 [==============================] - 6s 61ms/step - loss: 1.5816
Epoch 8/100
85/85 [==============================] - 6s 61ms/step - loss: 1.5256
Epoch 9/100
85/85 [==============================] - 6s 61ms/step - loss: 1.4742
Epoch 10/100
85/85 [==============================] - 6s 61ms/step - loss: 1.4268
Epoch 11/100
85/85 [==============================] - 6s 61ms/step - loss: 1.3852
Epoch 12/100
85/85 [==============================] - 6s 61ms/step - loss: 1.3491
Epoch 13/100
85/85 [=====

In [33]:
tf.train.latest_checkpoint(CHECKPOINT_DIR)

'/content/drive/My Drive/colab/seq2seq_divina/training_checkpoints/ckpt_100'

In [34]:
pred_model = build_model(len(alphabet), EMBEDDING_DIM, RNN_UNITS, batch_size=1)

pred_model.load_weights(tf.train.latest_checkpoint(CHECKPOINT_DIR))

pred_model.build(tf.TensorShape([1, None]))

In [35]:
pred_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19968     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 78)             79950     
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


In [36]:
def generate_text(model, 
                  start_string, 
                  temperature: float=1.0, 
                  num_generate: int=1000) -> str:

  """
    Evaluation step (generating text using the learned model)
    
    :param model:
      TensorFlow learned model

    :param start_string:
      Initial string to feed to model
    
    :param temperature:
      Low temperatures results in more predictable text.
      Higher temperatures results in more surprising text.
     
    :param num_generate:
      number of characters to generate

  """
  
  # Converting our start string to numbers (vectorizing)
  input_eval = [char2ind[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(ind2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [37]:
print(generate_text(pred_model, start_string=u"Nel mezzo del cammin", temperature=1.1))

Nel mezzo del cammin disio
dell'etterno, ch'era o alla cura:
quel primo Maria, di sé in odo, ove s'arresta,
avendo li occhi fuor cherci';
e io rimasi in via con esso i due
fu stadinanzi alli Anabbian conta».
  E come a colui che novità del duolo.
  Antandr c'hanno Italia morta,
sí come nuvoleggia,
e Faronici, non è l'altro caso e fera piú piú doglia
se, ralla veduta ui colora,
guardando le genti gloria e 'l Batista;
per che 'l mio viso in lei tutte men ch'io questo mond'io vivo,
era la mia virtú t'è in piacere?
L'acqua di Monsibil per piú petto.
  E tosto si vedrà di quel ch'io m'accuso
per escusarmi, e vedermi dir vero;
ché in te' nostro prede l'ardana e 'l triforme ai famigliarsi all'Espetto fissa gravidale ed olezzetta e terra,
e la voce ond'io parea bello
avvelson rabbiando,
per lo diletto diversi sol di sé piglio.
  Questi si percotean non può discittoso lume si dice
tua cogntullo spoglio,
  e «Ondo tu che, ma non arte, e parlar tu hoverchia,
sí che terra non potesse al suo Ce


In [38]:
print(generate_text(pred_model, start_string=u"Questi la caccera per ogni villa", temperature=1.1))

Questi la caccera per ogni villa,
fin che l'attese cortese oppinïone e m'accosa;
  ma poi ch'i' fui agorello, il nostri piedi:
miservi picciolse ancor di là non sia fosse quindi si leva.
  Fu colui che si si risponde,
  restare, voi bettiman nudo alla vita ria,
è Azzolator, che pur dar piú cara,
e comandò che l'amasseso quel frutto
che fa in nube il vero inver la cala;
  e io vi giugne, «or li canti, e poi tra lia
colui che muta
per l'aere tra Titoria tue».
  Ed elli a me: «Tu vero appresso io mi presta
ch'ella mi fece intrare appresso «il nido
a cui tanto possion dentro si duce:
  e vidi uscirci del suo dolce aspetto;
  ma perché piebbi si levò dalpretolato pone, felice, e Don è vivo,
avvolti, quando crea si pote da riversata,
  gridò: «Perché, se ben t'accostò al cerebre che porta?
ché, termine mi fe' l'E' miei passi
tocchendo chi si nascose».
  Ed elli a me: «Vano pensier che non ci ada la prima corno,
la poco di quinci e quindi la mira
esperighi mi or sí due noi ad una,
o per lo lc

In [39]:
pred_model.save(SAVE_DIR)

INFO:tensorflow:Assets written to: /content/drive/My Drive/colab/seq2seq_divina/saved_model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/colab/seq2seq_divina/saved_model/assets


In [40]:
new_model = tf.keras.models.load_model(SAVE_DIR, custom_objects={'loss': loss})

In [42]:
print(generate_text(new_model, start_string=u"Poi fui famiglia del buon re Tebaldo:"))

Poi fui famiglia del buon re Tebaldo:
pria col ciel col sol ti stringo».
  Quali i follini due che 'l sonno si spedí...
  O mostrò che l'arca li si partí di dolorosanti passai che 'l ciel non è stato;
però nel vi lasciato al cielo,
tra 'l suo fattor lo sangue splendori.
  L'altra è quella che leggi vicini.
  O mostrava cia del loco dov'io stava,
per avversari due perdonanzi»;
venimmo al punto dove si dibasta.
  Per per disiderie incontro a sé udito,
quei che la ripa, ch'era per la spigliarmi,
per lo 'nferno la Danuga,
  e visse, e rimaser lenti;
per che al cantar di là, ma perché sono
la possa in quanto vi trasplo,
com' credesti,
lasciala per non veggio in sí fatta on affetto duro scese.
  La donna mia agriglieo;
  ché, se so chi è piú notar m'appressai;
e 'l fummo del ruscel di sopra,
se un cielo a veder com'io veggio in su la roclittura,
corse rosplende,
  vid'io farsi quel segno, che di Silvo».
  Noi mi prescrise e la natural vedesse,
com'e' dissi, ch'al ciel diventa degno.
  Ma qui